In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats as spstats
from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
from sklearn.preprocessing import Imputer, LabelEncoder, PolynomialFeatures, StandardScaler, scale as skscale
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.externals import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

/Users/Meng/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
target = '标签'
uid = '申请编号'

def calc_auc(y_test, y_proba):
    auc = round(metrics.roc_auc_score(y_test, y_proba), 3)
    return auc

def ks_score(y_test, y_proba):
    scale = 4
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba, pos_label=1)
    KS = round(max(list(tpr-fpr)), scale)
    return KS

In [3]:
############## Load

# For kfold
n_folds = 5

# ### Test
# X = pd.read_csv('./tmp/train_d1234.csv', header=0, index_col=0).head(10000)
# X.shape

# y = pd.read_csv('./data/train_label.csv', header=0, index_col=0).head(10000)
# y.shape

''' Load '''
Xid = pd.read_csv('./tmp/train_d1234_nona.csv', header=0, index_col=0)
Xid.shape

yid = pd.read_csv('./data/train_label.csv', header=0, index_col=0)
yid.shape

''' Merge '''
# Merge
xy = pd.merge(Xid, yid, on=uid, how='inner')

# ''' Filter Bins ''' 
# uids = pd.read_csv('./tmp/train_uid0.csv', header=0, index_col=0)
# uids.columns = [uid]
# xy = pd.merge(uids, xy, on=uid, how='left')
# xy.shape

''' drop id'''
xy.drop(uid, axis=1, inplace=True)
xy.shape

''' Split '''
# X, y
X = xy.copy()
y = X.pop(target)
X.shape
y.shape

# ### List
# X = pd.read_csv('./tmp/train_d12_d3_dum.csv', header=0, index_col=0).drop(uid, axis=1).values
# len(X)
# len(X[0])

# y = pd.read_csv('./tmp/1_y.csv', header=0, index_col=0)['0'].values
# len(y)

' Load '

(140000, 812)

(140000, 1)

' Merge '

' drop id'

(140000, 812)

' Split '

(140000, 811)

(140000,)

In [4]:
# RF
param_fixed_rf = {
    'n_jobs' : -1,
    'oob_score' : True,
    'random_state':123,
    'verbose':0
}

# XGB
param_fixed_xgb = {
    'n_jobs' : -1,
    'eval_metric': 'auc',
    'seed' : 123,
    'silent' : 1,
    'verbose_eval':0
}

# LGB
param_fixed_lgb = {
    'n_jobs' : -1,
    'metric' : 'auc',
    'random_state' : 123,
    'bagging_seed':123,
    'feature_fraction_seed':123,
    'verbose_eval' : 0
}

# LR
param_fixed_lr = {
    'n_jobs' : -1,
    'random_state' : 123,
    'verbose' : 0     
}

In [6]:
class StackingModels (BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(self, base_models, base_feats, meta_model, if_feats_select, n_folds = n_folds):
        self.base_models = base_models
        self.meta_model = meta_model
        self.if_feats_select = if_feats_select
        # base_feats
        self.base_feats = []
        if self.if_feats_select == True:
            self.base_feats = base_feats
        self.n_folds = n_folds
        self.meta_X = []

    def fit(self, X_ttl, y):
        self.base_models_ = [list() for x in self.base_models]
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=123)
        
        # Get results of basic models
        out_of_fold_predictions = np.zeros((X_ttl.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            ### If select features
            if self.if_feats_select == True:
                X = X_ttl[self.base_feats[i]]
            else:
                X = X_ttl
            l = len(X.columns) # Feature count
            print(f'*** Base Model: {model.__class__}, Features: {l} ***')
            
            j = 0
            # Train
            for train_index, valid_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X.iloc[train_index],  y.iloc[train_index])
                y_pred = instance.predict_proba(X.iloc[valid_index])[:,1]
                ks = ks_score(y.iloc[valid_index], y_pred)
                print(ks)
                out_of_fold_predictions[valid_index, i] = y_pred
                j += 1

        # META
        self.meta_X = out_of_fold_predictions
        # Polynomial
        poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
        X = poly.fit_transform(out_of_fold_predictions)
        l = len(X[0]) # Feature Count
        print(f'*** Meta Model: {self.meta_model.__class__}, Features: {l} ***')
        j = 0
        self.meta_models_ = []
        for train_index, valid_index in kfold.split(X, y):
            instance = clone(self.meta_model)
            self.meta_models_.append(instance)
            instance.fit(X[train_index],  y[train_index])
            y_pred = instance.predict_proba(X[valid_index])[:,1]
            ks = ks_score(y[valid_index], y_pred)
            print(ks)
            j += 1
        return self

    # Prediect
    def predict(self, X_ttl, if_meta=True):
        
        # Features for Meta Model       
        meta_features = pd.DataFrame()
        for i, models_kfold in enumerate(self.base_models_):
            # If select features
            if self.if_feats_select == True:
                X = X_ttl[self.base_feats[i]]
            else:
                X = X_ttl
            # Predict base models
            prob_kfold = pd.DataFrame()
            for j, model in enumerate(models_kfold):
                prob_kfold[j] = model.predict_proba(X)[:,1]
            meta_features[i] = prob_kfold.mean(axis=1)
        
        # Prediect
        if if_meta:
            # Polynomial
            poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
            X = poly.fit_transform(meta_features)
            prob_kfold = pd.DataFrame()
            for j, model in enumerate(self.meta_models_):
                prob_kfold[j] = model.predict_proba(X)[:,1]
            pred = prob_kfold.mean(axis=1)
        else:
            pred = meta_features.mean(axis=1)
        return pred
    
############ Stacking

### Base models

# Base Featues
feats = np.load('./model/base_features.npy', allow_pickle=True).item()

# RF = RandomForestClassifier()
# XGB = XGBClassifier()
# LGB = LGBMClassifier()
# LR = LogisticRegression()

# RF
best_params_load = np.load('./model/base_rf.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_rf}
RF = RandomForestClassifier(**model_params)
# XGB
best_params_load = np.load('./model/base_xgb.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_xgb}
XGB = XGBClassifier(**model_params)
# LGB
best_params_load = np.load('./model/base_lgb.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_lgb}
LGB = LGBMClassifier(**model_params)
# LR
best_params_load = np.load('./model/base_lr.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_lr}
LR = LogisticRegression(**model_params)
# Mean
MEAN = 'MEAN'

### Stacking
base_models = [LGB, RF, XGB]
base_feats = [feats['lgb'], feats['rf'], feats['xgb']]
# base_models = [LGB, XGB]
# base_feats = [feats['lgb'], feats['xgb']]
meta_model = LR
CLF = StackingModels(base_models=base_models, meta_model=meta_model, base_feats=base_feats, if_feats_select=False)

# Fit
CLF.fit(X, y)

# # Predict
# pred = CLF.predict(X)
# ks_score(y, pred)

# # Save meta_X
# meta_X = pd.DataFrame(CLF.meta_X, columns=['lgb', 'rf', 'xgb'])
# # meta_X = pd.DataFrame(CLF.meta_X, columns=['lgb', 'xgb'])
# meta_X.shape
# meta_X.head()
# meta_X.to_csv('./model/x_meta.csv')

*** Base Model: <class 'lightgbm.sklearn.LGBMClassifier'>, Features: 811 ***
0.3716
0.3711
0.3528
0.3653
0.3532
*** Base Model: <class 'sklearn.ensemble.forest.RandomForestClassifier'>, Features: 811 ***
0.3116
0.302
0.3061
0.3066
0.3042
*** Base Model: <class 'xgboost.sklearn.XGBClassifier'>, Features: 811 ***
0.3581
0.3565
0.3484
0.35
0.353
*** Meta Model: <class 'sklearn.linear_model.logistic.LogisticRegression'>, Features: 6 ***
0.3733
0.3707
0.3533
0.3673
0.3557


StackingModels(base_feats=[],
               base_models=[LGBMClassifier(bagging_fraction=0.9, bagging_freq=4,
                                           bagging_seed=123,
                                           boosting_type='gbdt', cat_smooth=16,
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           feature_fraction=0.9,
                                           feature_fraction_seed=123,
                                           importance_type='split',
                                           is_unbalance=True,
                                           lambda_l1=0.8888888888888888,
                                           lambda_l2=25, learning_rate=0.02,
                                           max_depth=-1, met...
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=123, silent=1, subs

In [ ]:
# class StackingModels (BaseEstimator, ClassifierMixin, TransformerMixin):
#     def __init__(self, base_models, base_feats, meta_model, n_folds = n_folds):
#         self.base_models = base_models
#         self.base_feats = base_feats
#         self.meta_model = meta_model
#         self.n_folds = n_folds
#         self.meta_X = []

#     def fit(self, X_ttl, y):
#         self.base_models_ = [list() for x in self.base_models]
#         kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=123)
        
#         # Get results of basic models
#         out_of_fold_predictions = np.zeros((X_ttl.shape[0], len(self.base_models)))
#         for i, model in enumerate(self.base_models):
#             l = len(self.base_feats[i]) # Feature Count
#             print(f'*** Base Model: {model.__class__}, Features: {l} ***')
#             j = 0
#             X = X_ttl[self.base_feats[i]]
#             for train_index, valid_index in kfold.split(X, y):
#                 instance = clone(model)
#                 self.base_models_[i].append(instance)
#                 instance.fit(X.iloc[train_index],  y.iloc[train_index])
#                 y_pred = instance.predict_proba(X.iloc[valid_index])[:,1]
#                 ks = ks_score(y.iloc[valid_index], y_pred)
#                 print(ks)
#                 out_of_fold_predictions[valid_index, i] = y_pred
#                 j += 1

#         # META
#         self.meta_X = out_of_fold_predictions
#         # Polynomial
#         poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
#         X = poly.fit_transform(out_of_fold_predictions)
#         l = len(X[0]) # Feature Count
#         print(f'*** Meta Model: {self.meta_model.__class__}, Features: {l} ***')
#         j = 0
#         self.meta_models_ = []
#         for train_index, valid_index in kfold.split(X, y):
#             instance = clone(self.meta_model)
#             self.meta_models_.append(instance)
#             instance.fit(X[train_index],  y[train_index])
#             y_pred = instance.predict_proba(X[valid_index])[:,1]
#             ks = ks_score(y[valid_index], y_pred)
#             print(ks)
#             j += 1
#         return self

#     # Prediect
#     def predict(self, X_ttl, if_meta=True):
        
#         # Features for Meta Model       
#         meta_features = pd.DataFrame()
#         for i, models_kfold in enumerate(self.base_models_):
#             X = X_ttl[self.base_feats[i]] 
#             prob_kfold = pd.DataFrame()
#             for j, model in enumerate(models_kfold):
#                 prob_kfold[j] = model.predict_proba(X)[:,1]
#             meta_features[i] = prob_kfold.mean(axis=1)
        
#         # Prediect
#         if if_meta:
#             # Polynomial
#             poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
#             X = poly.fit_transform(meta_features)
#             prob_kfold = pd.DataFrame()
#             for j, model in enumerate(self.meta_models_):
#                 prob_kfold[j] = model.predict_proba(X)[:,1]
#             pred = prob_kfold.mean(axis=1)
#         else:
#             pred = meta_features.mean(axis=1)
#         return pred
    
# ############ Stacking

# ### Base models

# # Base Featues
# feats = np.load('./model/base_features.npy', allow_pickle=True).item()

# # RF = RandomForestClassifier()
# # XGB = XGBClassifier()
# # LGB = LGBMClassifier()
# # LR = LogisticRegression()

# # RF
# best_params_load = np.load('./model/base_rf.npy', allow_pickle=True).item()
# model_params = {**best_params_load, **param_fixed_rf}
# RF = RandomForestClassifier(**model_params)
# # XGB
# best_params_load = np.load('./model/base_xgb.npy', allow_pickle=True).item()
# model_params = {**best_params_load, **param_fixed_xgb}
# XGB = XGBClassifier(**model_params)
# # LGB
# best_params_load = np.load('./model/base_lgb.npy', allow_pickle=True).item()
# model_params = {**best_params_load, **param_fixed_lgb}
# LGB = LGBMClassifier(**model_params)
# # LR
# best_params_load = np.load('./model/base_lr.npy', allow_pickle=True).item()
# model_params = {**best_params_load, **param_fixed_lr}
# LR = LogisticRegression(**model_params)
# # Mean
# MEAN = 'MEAN'

# ### Stacking
# base_models = [LGB, RF, XGB]
# base_feats = [feats['lgb'], feats['rf'], feats['xgb']]
# # base_models = [LGB, XGB]
# # base_feats = [feats['lgb'], feats['xgb']]
# meta_model = LR
# CLF = StackingModels(base_models=base_models, base_feats=base_feats, meta_model=meta_model)

# # Fit
# CLF.fit(X, y)

# # # Predict
# # pred = CLF.predict(X)
# # ks_score(y, pred)

# # # Save meta_X
# # meta_X = pd.DataFrame(CLF.meta_X, columns=['lgb', 'rf', 'xgb'])
# # # meta_X = pd.DataFrame(CLF.meta_X, columns=['lgb', 'xgb'])
# # meta_X.shape
# # meta_X.head()
# # meta_X.to_csv('./model/x_meta.csv')

In [ ]:
############## Test

def norm_score(score):
    if score < 0:
        score = 0
    elif score > 1:
        score = 1
    return score

test = pd.read_csv('./tmp/A_d1234.csv', header=0, index_col=0)
X_test = test.drop(uid, axis=1)
X_test.shape

# Predict
scores = CLF.predict(X_test)

# Output
uids = test[uid]
out = pd.concat([uids, pd.Series(scores)], axis=1)
out[0] = out[0].apply(norm_score)
out.head()
out.to_csv('./model/predict.csv', header=False, index=False)

In [ ]:
# # ############## Predict Train

# def norm_score(score):
#     if score < 0:
#         score = 0
#     elif score > 1:
#         score = 1
#     return score

# # Predict
# scores = CLF.predict(Xid)

# # Output
# uids = Xid[uid]
# out = pd.concat([uids, pd.Series(scores)], axis=1)
# out[0] = out[0].apply(norm_score)
# out.head()
# out.to_csv('./model/predict_train.csv')

In [ ]:
# ####### Distributions

# # Final
# sns.distplot(pred)
# sns.distplot(scores)
# plt.show()

# meta_X = CLF.meta_X
# meta_X.shape
# meta_X[0]

# # Base
# for i in range(3):
#     i
#     sns.distplot(meta_X[:,i])

In [ ]:
# ############## Ana

# base_models = [LGB, RF, XGB]
# base_models_ = [list() for x in base_models]
# kfold = KFold(n_splits=2, shuffle=True, random_state=123)

# out_of_fold_predictions = np.zeros((X.shape[0], len(base_models)))
# for i, model in enumerate(base_models):
#     j = 0
#     for train_index, valid_index in kfold.split(X, y):
#         instance = clone(model)
#         base_models_[i].append(instance)
#         instance.fit(X.loc[train_index],  y.loc[train_index])
#         len(X.loc[train_index])
#         y_pred = instance.predict_proba(X.loc[valid_index])[:,1]
#         ks = ks_score(y.loc[valid_index], y_pred)
#         print(f'* KS({j}):{ks} *')
#         out_of_fold_predictions[valid_index, i] = y_pred
#         j += 1

In [ ]:
# len(out_of_fold_predictions)
# len(out_of_fold_predictions[0])
# out_of_fold_predictions[0]

In [ ]:
# meta_model = LR
# meta_model_ = clone(meta_model)

# meta_model_.fit(out_of_fold_predictions, y)
# y_pred = meta_model_.predict_proba(out_of_fold_predictions)[:,1]
# ks_score(y, y_pred)

In [ ]:
# meta_features = np.column_stack ([
#     np.column_stack(
#         [model.predict_proba(X)[:,1] for model in base_models]
#     ).mean (axis=1)
#     for base_models in base_models_])

In [ ]:
# len(meta_features)
# len(meta_features[0])
# meta_features[0]

In [ ]:
# meta_features = pd.DataFrame()
# for i, models_kfold in enumerate(base_models_):
#     i
#     prob_kfold = pd.DataFrame()
#     for j, model in enumerate(models_kfold):
#         j
#         prob_kfold[j] = model.predict_proba(X)[:,1]
#         prob_kfold.shape
#     prob_kfold.head()
#     meta_features[i] = prob_kfold.mean(axis=1)

In [ ]:
# meta_features.shape
# meta_features.head()

In [ ]:
# ''' RF '''
# RF.fit(meta_features, y)
# ks_score(y, RF.predict_proba(meta_features)[:,1])

# ''' RF1 '''
# RF1 = RandomForestClassifier()
# RF1.fit(meta_features, y)
# ks_score(y, RF1.predict_proba(meta_features)[:,1])

# ''' LGB '''
# LGB.fit(meta_features, y)
# ks_score(y, LGB.predict_proba(meta_features)[:,1])

# ''' LR '''
# LR.fit(meta_features, y)
# ks_score(y, LR.predict_proba(meta_features)[:,1])

# ''' MEAN '''
# # LR.fit(meta_features, y)
# ks_score(y, meta_features.mean(axis=1))